<a href="https://colab.research.google.com/github/YoheiFukuhara/recommender-system/blob/main/04_userbased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import datetime
import pickle

from sortedcontainers import SortedList
from google.colab import drive
import numpy as np

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
BASE_PATH = '/content/drive/MyDrive/ColabNotebooks/ML/Recommend/output/'

In [ ]:
def load_pickle(file):
    with open(BASE_PATH+file+'.pickle', 'rb') as f:
        return pickle.load(f)

user2movie = load_pickle('user2movie')
movie2user = load_pickle('movie2user')
usermovie2rating = load_pickle('usermovie2rating')
usermovie2rating_test = load_pickle('usermovie2rating_test')

In [ ]:
N = np.max(list(user2movie.keys())) + 1
m1 = np.max(list(movie2user.keys()))
m2 = np.max([m for (u, m), r in usermovie2rating_test.items()])
M = max(m1, m2) + 1
print('N:', N, 'M:', M)  #if N is more than 10000, it takes so much time to process.

N: 4000 M: 1000


to find the user similarities, you have to do O(N^2 * M) calculations!
in the "real-world" you'd want to parallelize this
note: we really only have to do half the calculations, since w_ij is symmetric

In [ ]:
%%time
K = 25 # number of neighbors we'd like to consider
limit = 5 # number of common movies users must have in common in order to consider
neighbors = [] # store neighbors in this list
averages = [] # each user's average rating for later use
deviations = [] # each user's deviation for later use

def calc_user_data(n):
    movies_n_set = set(user2movie[n])

   # ユーザの各rating
    ratings = { movie: usermovie2rating[(n, movie)] for movie in user2movie[n] }
 
    # ユーザの平均Rating
    avg = np.mean(list(ratings.values()))  

    # ユーザの平均Rating と実Ratingとの差 の一覧
    dev = {movie: (rating - avg) for movie, rating in ratings.items()} 

    # ユーザの平均Rating と実Ratingとの差 の一覧(Movie情報なし)
    dev_values = np.array(list(dev.values()))

    # ベクトル内積を計算することで偏差を出力
    # https://www.yukisako.xyz/entry/correlation-coefficient
    sigma = np.sqrt(dev_values.dot(dev_values))

    return movies_n_set, avg, dev, sigma


for i in range(N):
    movies_i_set, avg_i, dev_i, sigma_i = calc_user_data(i)

    averages.append(avg_i)
    deviations.append(dev_i)

    sl = SortedList()

    # 対称的データなので、計算量を半分にできるが、半分にせずにアルゴリズムを簡略化
    for j in range(N):
        if j != i:
            common_movies = (movies_i_set & set(user2movie[j])) # 和集合
            if len(common_movies) > limit:
                
                _, avg_j, dev_j, sigma_j = calc_user_data(j)
                
                # 相関係数算出
                w_ij = sum(dev_i[m]*dev_j[m] for m in common_movies) / (sigma_i * sigma_j)

                # insert into sorted list and truncate
                # negate weight, because list is sorted ascending
                # maximum value (1) is "closest"
                sl.add((-w_ij, j))

                # しきい値Kを超過していたら末尾(最低値)を削除
                if len(sl) > K:
                    del sl[-1]
    
    neighbors.append(sl)
    print("\r{0}".format(i), end="")

#    if i % 1000 == 0:        
#        print(f'{i}: {datetime.datetime.now()}')

3999CPU times: user 3h 15min 35s, sys: 3min 11s, total: 3h 18min 46s
Wall time: 3h 12min 35s


In [ ]:
neighbors

[SortedList([(-0.4925209027297742, 620), (-0.41517533288090425, 1), (-0.3957598369163864, 193), (-0.3889190923207677, 18), (-0.380195770780848, 99), (-0.37896190901210414, 91), (-0.37557927149419446, 118), (-0.37516699590947444, 44), (-0.3726628961090899, 505), (-0.3713111763100667, 570), (-0.36557291421701316, 483), (-0.3635727191536178, 493), (-0.3630057813714799, 1931), (-0.3610921940382752, 977), (-0.3584564226225422, 3071), (-0.35761505383952014, 141), (-0.3525841972427348, 71), (-0.35205056317102157, 43), (-0.35110185981689934, 54), (-0.35105417026646774, 24), (-0.35031244518191496, 290), (-0.34998654475278784, 680), (-0.34933242821671856, 1108), (-0.3493234646981278, 407), (-0.34752297415946787, 444)]),
 SortedList([(-0.6861605123257166, 620), (-0.596134738716517, 2542), (-0.5752562431045296, 1906), (-0.5501621467743583, 26), (-0.5458334682846799, 69), (-0.5217426983749682, 141), (-0.520439118450131, 3014), (-0.5128282932331445, 103), (-0.5114530612981157, 164), (-0.510091155754

In [ ]:
def predict(i, m):
    # calculate the weighted sum of deviations
    numerator = 0
    denominator = 0

    # k近傍のj読込
    for neg_w, j in neighbors[i]:
        # remember, the weight is stored as its negative
        # so the negative of the negative weight is the positive weight
        try:
            numerator += -neg_w * deviations[j][m] # 分子
            denominator += abs(neg_w)
        except KeyError:
      # neighbor may not have rated the same movie
      # don't want to do dictionary lookup twice
      # so just throw exception
            pass

    if denominator == 0:
        prediction = averages[i]
    else:
        prediction = numerator / denominator + averages[i]
    prediction = min(5, prediction)
    prediction = max(0.5, prediction) # min rating is 0.5
    return prediction

In [ ]:
%%time
train_predictions = []
train_targets = []
for (i, m), target in usermovie2rating.items():
    # calculate the prediction for this movie
    prediction = predict(i, m)

  # save the prediction and target
    train_predictions.append(prediction)
    train_targets.append(target)

CPU times: user 1min, sys: 103 ms, total: 1min
Wall time: 1min


In [ ]:
%%time
test_predictions = []
test_targets = []
# same thing for test set
for (i, m), target in usermovie2rating_test.items():
    # calculate the prediction for this movie
    prediction = predict(i, m)

  # save the prediction and target
    test_predictions.append(prediction)
    test_targets.append(target)

CPU times: user 15.1 s, sys: 32.4 ms, total: 15.1 s
Wall time: 15.1 s


In [ ]:
# calculate accuracy
def mse(p, t):
    p = np.array(p)
    t = np.array(t)
    return np.mean((p - t)**2)

print('train mse:', mse(train_predictions, train_targets))
print('test mse:', mse(test_predictions, test_targets))

train mse: 0.5391769565502138
test mse: 0.6171277075062406
